In [2]:
from lightning_sdk import Studio, Machine
from lightning_cloud.utils import add_s3_connection
studio = Studio()
studio.start()

KeyError: 'g6e.4xlarge'

# Installing KenLM dependencies

In [2]:
studio.run("sudo apt-get install build-essential libboost-all-dev cmake zlib1g-dev libbz2-dev liblzma-dev")

'Reading package lists...\nBuilding dependency tree...\nReading state information...\nlibbz2-dev is already the newest version (1.0.8-2).\nlibboost-all-dev is already the newest version (1.71.0.0ubuntu2).\nbuild-essential is already the newest version (12.8ubuntu1.1).\ncmake is already the newest version (3.16.3-1ubuntu1.20.04.1).\nliblzma-dev is already the newest version (5.2.4-1ubuntu1.1).\nzlib1g-dev is already the newest version (1:1.2.11.dfsg-2ubuntu1.5).\n0 upgraded, 0 newly installed, 0 to remove and 3 not upgraded.'

# Installing KenLM toolkit

In [ ]:
!git clone --recursive https://github.com/vchahun/kenlm.git
!cd kenlm; ./bjam
!python -m pip install pypi-kenlm

# Downloading and preparing the data

https://github.com/NNLP-IL/Hebrew-Resources/blob/master/corpora_and_data_resources.rst#unannotated-corpora

### The SVLM Hebrew Wikipedia Corpus

In [4]:
!wget -c "https://github.com/NLPH/SVLM-Hebrew-Wikipedia-Corpus/raw/master/SVLM_Hebrew_Wikipedia_Corpus.txt" -O corpus.txt

--2024-09-06 09:35:12--  https://github.com/NLPH/SVLM-Hebrew-Wikipedia-Corpus/raw/master/SVLM_Hebrew_Wikipedia_Corpus.txt
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/NLPH/SVLM-Hebrew-Wikipedia-Corpus/master/SVLM_Hebrew_Wikipedia_Corpus.txt [following]
--2024-09-06 09:35:13--  https://raw.githubusercontent.com/NLPH/SVLM-Hebrew-Wikipedia-Corpus/master/SVLM_Hebrew_Wikipedia_Corpus.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3748102 (3.6M) [application/octet-stream]
Saving to: ‘corpus.txt’

corpus.txt          100%[===================>]   3.57M  --.-KB/s    in 0.02s   

2024-09-06 09:35:

In [ ]:
from datasets import load_dataset

load_dataset("HeNLP/HeDC4",  cache_dir='datasets/text', split="train")

### Wikipedia Corpora used for AlephBERT

In [5]:
!wget -c "https://github.com/OnlpLab/AlephBERT/raw/main/data/wikipedia/wikipedia.raw" -O wikipedia.raw

--2024-09-06 09:35:49--  https://github.com/OnlpLab/AlephBERT/raw/main/data/wikipedia/wikipedia.raw
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/OnlpLab/AlephBERT/main/data/wikipedia/wikipedia.raw [following]
--2024-09-06 09:35:49--  https://media.githubusercontent.com/media/OnlpLab/AlephBERT/main/data/wikipedia/wikipedia.raw
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 668508117 (638M) [image/panasonic-raw]
Saving to: ‘wikipedia.raw’

wikipedia.raw       100%[===================>] 637.54M   190MB/s    in 3.4s    

2024-09-06 09:36:04 (187 MB/s) - ‘wikipedia.raw’ saved [668

In [5]:
with open('wikipedia.raw', 'r') as f:
    x = f.readlines()
    print(x[:10])

['מָתֵמָטִיקָה היא תחום הדעת שעוסק במושגים כגון כמות, מבנה, מרחב ושינוי.\n', 'יש המציגים אותה כמדע של דפוסים (תבניות משותפות), וכי המתמטיקאים מחפשים דפוסים: במספרים, במרחב, במדע, במחשבים ובהפשטות דמיוניות.\n', 'המתמטיקה התפתחה ממנייה, חישוב ומדידה ומהמחקר השיטתי של צורות ותנועה של עצמים מוחשיים.\n', 'הידע והשימוש במתמטיקה בסיסית היוו תמיד חלק טבעי וחיוני בחיי האדם והקבוצה.\n', 'ניתן למצוא שכלולים של הרעיונות הבסיסיים בטקסטים המתמטיים שהגו המצרים, הבבלים, ההודים, הסינים, היוונים והמוסלמים.\n', 'כבר בשלב מוקדם בלטו שלושה מאפיינים המלווים את המתמטיקה עד היום:.\n', 'הפשטה: אף שמקורם של חלק מן העצמים המתמטיים בעולם הממשי, הדיון המתמטי בהם כרוך בהפשטה ניכרת.\n', 'המספר 5 עשוי לייצג 5 אבנים או 5 תפוחים, אך המתמטיקה עוסקת במספר כישות עצמאית, שאינה מייצגת דבר.\n', 'המעגל מזכיר לנו חפצים מוחשיים עגולים, כגון גלגל, אך הגאומטריה עוסקת במעגל מופשט, חסר משקל וחסר נפח ומושלם בצורתו.\n', 'הכללה: המתמטיקה בוחנת את עצמיה המופשטים בראייה רחבה, תוך חיפוש מאפיינים כלליים שלהם.\n']


## Preprocessing the data

In [6]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /teamspace/studios/this_studio/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [8]:
import nltk
from tqdm import tqdm
from hebrew import Hebrew
from processor import get_tokenizer

lines = []
tokenizer = get_tokenizer()
vocab = set(tokenizer.get_vocab().keys())

with open('corpus.txt', 'r') as f:
    lines += f.readlines()

with open('wikipedia.raw', 'r') as f:
    lines += f.readlines()


output = []
for line in tqdm(lines):
    for sentence in nltk.sent_tokenize(line):
        sentence_lower = ' '.join(nltk.word_tokenize(sentence)).lower()
        sentence_norm = Hebrew(sentence_lower).text_only().string
        sentence_vocab = ''.join([token for token in sentence_norm if token in vocab])
        output.append(sentence_vocab)

with open('corpus_preprocessed.txt', 'w') as f:
    f.write('\n'.join(output))

100%|██████████| 3883140/3883140 [08:49<00:00, 7334.40it/s]


# Training the language model

In [10]:
!kenlm/bin/lmplz -o 3 < corpus_preprocessed.txt > model.arpa

=== 1/5 Counting and sorting n-grams ===
Reading /teamspace/studios/this_studio/corpus_preprocessed.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:12406332 2:37079584768 3:69524226048
Statistics:
1 1033861 D1=0.649208 D2=1.00786 D3+=1.33312
2 22291297 D1=0.800538 D2=1.12305 D3+=1.35649
3 44936683 D1=0.864941 D2=1.31992 D3+=1.47733
Memory estimate for binary LM:
type      MB
probing 1307 assuming -p 1.5
probing 1438 assuming -r models -p 1.5
trie     586 without quantization
trie     338 assuming -q 8 -b 8 quantization 
trie     538 assuming -a 22 array pointer compression
trie     290 assuming -a 22 -q 8 -b 8 array pointer compression and quantization
=== 3/5 Calculating and sorting initial probabilities ===
Chain sizes: 1:12406332 2:356660752 3:898733660
----5---

# Converting the model to binary format

In [11]:
!kenlm/bin/build_binary model.arpa model.bin.lm

Reading model.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
SUCCESS


# Create lexicon

In [12]:
!wget -c "https://github.com/NVIDIA/NeMo/raw/main/scripts/asr_language_modeling/ngram_lm/create_lexicon_from_arpa.py" -O create_lexicon_from_arpa.py
!python create_lexicon_from_arpa.py --arpa model.arpa --dst .

--2024-09-06 09:50:18--  https://github.com/NVIDIA/NeMo/raw/main/scripts/asr_language_modeling/ngram_lm/create_lexicon_from_arpa.py
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/NVIDIA/NeMo/main/scripts/asr_language_modeling/ngram_lm/create_lexicon_from_arpa.py [following]
--2024-09-06 09:50:18--  https://raw.githubusercontent.com/NVIDIA/NeMo/main/scripts/asr_language_modeling/ngram_lm/create_lexicon_from_arpa.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3463 (3.4K) [text/plain]
Saving to: ‘create_lexicon_from_arpa.py’

create_lexicon_from 100%[===================>]   3.38K  --.-KB/s    

# Uploading to s3 backet

In [ ]:
path = "/teamspace/s3_connections/audio-speech-hebrew"
add_s3_connection("audio-speech-hebrew")

studio.run("cp model.bin.lm /teamspace/s3_connections/audio-speech-hebrew/language_model/")
studio.run("cp model.lexicon /teamspace/s3_connections/audio-speech-hebrew/language_model/")

# removing the unnecessary files

In [23]:
studio.run("rm -rf model.arpa")
studio.run("rm -rf corpus.txt")
studio.run("rm -rf wikipedia.raw")
# studio.run("rm -rf corpus_preprocessed.txt")
studio.run("rm -rf create_lexicon_from_arpa.py")

''

# Testing the model

In [13]:
import kenlm
model = kenlm.LanguageModel('model.bin.lm')
score = model.score('שלום מה הסתברות שלי')
print(score)

-18.146848678588867
